# ETL Notebook

This notebook is to do some DataFrame visualisation for our ETL process. We will run connect to the Database and run our SQL queries to extract our defined dimensions.

This can also be our testbed for potential transforms that we will be conducting for our NLP models.

In [53]:
# Import libraries
import mysql.connector
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
from uuid import uuid4
load_dotenv()

True

In [54]:
db_host = os.getenv("DATABASE_ENDPOINT")
db_user = os.getenv("DATABASE_USERNAME")
db_pw = os.getenv("DATABASE_PASSWORD")
db_name = os.getenv("DATABASE_NAME")
db_port = os.getenv("DATABASE_PORT")

dwh_host = os.getenv("DATAWH_ENDPOINT")
dwh_user = os.getenv("DATAWH_USERNAME")
dwh_pw = os.getenv("DATAWH_PASSWORD")
dwh_name = os.getenv("DATAWH_NAME")
dwh_port = os.getenv("DATAWH_PORT")


db_tripadvisor = mysql.connector.connect(
	host=db_host,
	user=db_user,
	passwd=db_pw,
	database=db_name
)

engine = create_engine(f'mysql://{dwh_user}:{dwh_pw}@{dwh_host}:{dwh_port}/{dwh_name}', echo=False)

dwh = engine.connect()

In [55]:
# Check tripadvisor_reviews table

str_sql = '''
SELECT *
FROM tripadvisor_reviews
'''

df = pd.read_sql(sql=str_sql, con=db_tripadvisor)
df

/var/folders/0m/1wpwx38n6tdb_9y7mx9pvb_c0000gn/T/ipykernel_46333/3910135080.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=str_sql, con=db_tripadvisor)


,Review Title,Review Text,Date of Stay,Author Contribution,Rating
0,Must see in Singapore,A must not miss place for tourists to visit wh...,2024-03-01,73,5.0
1,Marina Bay world class,Amazing hotel and loved the facilities. Being ...,2024-03-01,1,5.0
2,Nice touch.,While the initial check in experience was not ...,2024-03-01,2,5.0
3,Amazing experience at MBS with superb hospital...,I booked the Sands Premiere Suite to celebrate...,2024-03-01,1,5.0
4,Incredibile,"Nina is amazing as all the team, I loved her h...",2024-03-01,2,5.0
...,...,...,...,...,...
11227,Aweseome view over the Harbour,Amazing design ! I wouldnt recommend to sleep...,2014-03-01,131,4.0
11228,Excellent Property but poor services,Stayed in this hotel during the period Novembe...,2014-03-01,15,3.0
11229,Stunning &amp; Mesmeric,When you walk out on to that roof for the firs...,2014-03-01,25,5.0
11230,A Slice of Heaven,One can say what they want about this property...,2014-02-01,5,4.0


In [56]:
# Time dimension
# TimeID
# StayDate
# StayDateYear
# StayDateMonth
# StayDateDay
# StayDateDayOfWeek
# StayDateWeek

time_sql = """
SELECT `Date of Stay` as StayDate, YEAR(`Date of Stay`) AS StayDateYear, MONTH(`Date of Stay`) AS StayDateMonth, Day(`Date of Stay`) AS StayDateDay, IF((DayOfWeek(`Date of Stay`) - 1) = 0, 7, DayOfWeek(`Date of Stay`) - 1) As StayDateDayOfWeek, WEEK(`Date of Stay`) AS StayDateWeek
FROM tripadvisor_reviews;
"""

df = pd.read_sql(sql=time_sql, con=db_tripadvisor)
df['TimeID'] = df['StayDate'].apply(lambda x: uuid4())
cols = df.columns.to_list()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df

/var/folders/0m/1wpwx38n6tdb_9y7mx9pvb_c0000gn/T/ipykernel_46333/2737257725.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=time_sql, con=db_tripadvisor)


,TimeID,StayDate,StayDateYear,StayDateMonth,StayDateDay,StayDateDayOfWeek,StayDateWeek
0,7b52e801-8d17-492b-a973-94e3f6110e72,2024-03-01,2024,3,1,5,8
1,7738df00-8cbc-4efa-8a60-8217f88b75b2,2024-03-01,2024,3,1,5,8
2,3d4d4d2a-29e8-4f9f-ba9a-8dc55aceb044,2024-03-01,2024,3,1,5,8
3,6234316a-463e-4a1c-a51d-1ee49170c3d8,2024-03-01,2024,3,1,5,8
4,476274b6-909c-4ef1-b587-ed3510923888,2024-03-01,2024,3,1,5,8
...,...,...,...,...,...,...,...
11227,d457a506-4e13-4279-8392-88906835df57,2014-03-01,2014,3,1,6,8
11228,c5072179-18ac-424b-a477-3a471400cc58,2014-03-01,2014,3,1,6,8
11229,15969497-12c9-41fe-a16e-107eba63e578,2014-03-01,2014,3,1,6,8
11230,b186a48c-f0f9-4bb9-80e0-3ae092b7e8e9,2014-02-01,2014,2,1,6,4


In [ ]:
# Review Dimension
# ReviewID
# ReviewText
# ReviewTitle

